In [8]:
import numpy as np
import pandas as pd
from cs50 import SQL
from scipy.stats import rv_discrete
from datetime import datetime

db = SQL("sqlite:///qatarwc.db")

In [2]:
class Team:
    """Football team in World Cup"""
    def __init__(self, name, group):
        self.name = name
        self.group = group
        self.points = 0
        self.goals_scored = 0
        self.goals_received = 0
        self.stage = "groups"

    def match(self, scored, received):
        self.goals_scored += scored
        self.goals_received += received
        # Update points depending on score
        self.points += score2pts(scored, received)

    def eliminate(self):
        self.stage = 'eliminated'

In [3]:
def create_teams(): 
    """Loads teams from db into Team objects."""
    # Dict of Team instances
    TEAMS = {}   
    for i, team in enumerate(db.execute('SELECT code, "group" FROM teams;')):
        TEAMS[team['code']] = Team(team['code'], team['group'])
    return TEAMS

def score2pts(scored, received):
    """Given a score, returns the number of poins for one team."""
    if scored > received:
            return 3
    elif scored == received:
            return 1
    else:
        return 0

def simulate_score():
    """
    Returns the final score of a simulated football match. 
    The max goals ever scored in a WC match are 12.
    The max goals scored by one team are 10.
    """
    # Custom probability distribution of total match goals
    xk = range(13)
    pk = (0.09, 0.18, 0.28, 0.24, 0.09, 0.065, 0.025, 0.015, 0.0065, 0.004, 0.002, 0.0015, 0.001)
    pdist = rv_discrete(values=(xk, pk))

    # Generate sample
    total_goals = pdist.rvs() 
    # Assign a portion of goals to one team (max 10)
    t1_goals = np.random.randint(total_goals%10,11) if total_goals>10 else np.random.randint(0, total_goals + 1)

    # Return score
    return t1_goals, total_goals - t1_goals

In [42]:
def simulate_group_stage(TEAMS):
    """"
    Simulates all the group stage matches according to FIFA rules.
    https://digitalhub.fifa.com/m/2744a0a5e3ded185/original/FIFA-World-Cup-Qatar-2022-Regulations_EN.pdf
    """
    group_matches = db.execute("SELECT match, team1, team2 FROM fixtures WHERE stage = 'group matches';")
    groups_df = pd.DataFrame(group_matches).set_index('match')

    t1_goals=[]
    t2_goals=[]
    for _, row in groups_df.iterrows():
        g1, g2 = simulate_score()
        # Store score
        t1_goals.append(g1)
        t2_goals.append(g2)
        # Update statistics of both teams
        TEAMS[row['team1']].match(g1, g2)
        TEAMS[row['team2']].match(g2, g1)

    groups_df.insert(2, 't1_goals', t1_goals)
    groups_df.insert(3, 't2_goals', t2_goals)

    return(groups_df)

def get_group_rank(label, team_names, TEAMS, fixtures):
    """
    Obtains the ranking of a given group according to FIFA rules:
    https://digitalhub.fifa.com/m/2744a0a5e3ded185/original/FIFA-World-Cup-Qatar-2022-Regulations_EN.pdf
    Return a dict of position:team and the criteria used for tie-break (if necessary)
    """
    
    # Create a list of relevant group stats
    group_stats = list()
    for team in team_names:
        t = TEAMS[team]
        group_stats.append([t.name, t.points, t.goals_scored - t.goals_received, t.goals_scored])
        
    # Order teams by pts, gdf and gs (CRITERIA (a)-(c))
    group_df = pd.DataFrame(group_stats, columns=['team','pts','gdf', 'gs']).sort_values(['pts','gdf','gs'], ascending=[False, False, False])
    group_df.index = [1,2,3,4]
    #print(group_df)
    
    # Check if two or more teams (out of the first 3) are still tied
    dups = group_df.duplicated(subset=['pts','gdf','gs'], keep=False)
    is_tied = dups[0:3].sum()
    if is_tied <= 1:
        # No ties
        return group_df[['team']].to_dict()['team'], 'a-c'
    
    # Consider matches among tied teams
    tied_teams = group_df[dups]['team'].to_list()
    tiebreak = dict() # Stores pts, gdf and gs only from those mathces (CRITERIA (d)-(f))
    
    if group_df[dups].drop_duplicates(subset=['pts','gdf','gs']).shape[0] != 1:
    # If there are two pairs of tied teams (1&2 and 3&4) we only care about 1&2
        tied_teams = tied_teams[:2]
        
    for match in fixtures:
        # Select matches of interest
        if match['t1'] in tied_teams and match['t2'] in tied_teams:
            # Store stats
            g1, g2 = match['t1_goals'], match['t2_goals']
            if match['t1'] not in tiebreak:
                tiebreak[match['t1']] = np.asarray([score2pts(g1, g2), g1-g2, g1])
            else:
                tiebreak[match['t1']] += np.asarray([score2pts(g1, g2), g1-g2, g1])

            if match['t2'] not in tiebreak:
                tiebreak[match['t2']] = np.asarray([score2pts(g2, g1), g2-g1, g2])
            else:
                tiebreak[match['t2']] += np.asarray([score2pts(g2, g1), g2-g1, g2])
            #print(match)
            
    # Order tied teams 
    tbreak = pd.DataFrame.from_dict(tiebreak, orient='index', columns=['pts','gdf', 'gs']).sort_values(['pts','gdf','gs'], ascending=[False, False, False])
    #print(tbreak)
    
    # Rebuilt table of positions after tie-break
    positions=dict()
    j=0
    for i in range(1,5):
        if dups[i] == True:
            positions[i] = tbreak.index[j]
            j += 1
        else:
            positions[i] = group_df.iloc[i-1]['team']
    
    # If there are still ties the remaining criteria are not handled here (CRITERIA (g) & (h)))
    dups = tbreak.duplicated(subset=['pts','gdf','gs'], keep=False)
    is_tied = dups.sum()
    
    if is_tied <= 0:
        return positions, 'd-f'

    else:
        return positions, 'g-h'

In [6]:
group_labels = db.execute('SELECT DISTINCT "group" FROM teams') # Returns a list of dicts
GROUPS = [list(d.values())[0] for d in group_labels]
GROUPS

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [47]:
TEAMS = create_teams()
# Organize teams and fixtures by groups
group_teams = dict()
group_fixtures = dict()
for g in GROUPS:
    teams = db.execute('SELECT code FROM teams WHERE "group"=?', g)
    group_teams[g] = [team['code'] for team in teams]

    fixtures = db.execute('SELECT * FROM fixtures WHERE team1 IN (SELECT code FROM teams WHERE "group"=?) ORDER BY date;', g)
    # Change date formant e.g. 2022-12-02 into "Dec 02"
    group_fixtures[g] = [{'date': datetime.strptime(match['date'], "%Y-%m-%d").strftime("%b %d"),
                         'id':match['match'], 't1': match['team1'], 't2' :match['team2'], 
                         't1_goals': '',
                         't2_goals': ''} for match in fixtures]

scores = simulate_group_stage(TEAMS).to_dict('index')
g_sim = dict()
for g in GROUPS:
    fixtures = db.execute('SELECT * FROM fixtures WHERE team1 IN (SELECT code FROM teams WHERE "group"=?) ORDER BY date;', g)
    for i, match in enumerate(fixtures):
        group_fixtures[g][i]['t1_goals'] = int(scores[match['match']]['t1_goals'])
        group_fixtures[g][i]['t2_goals'] = int(scores[match['match']]['t2_goals'])

    g_sim[g] = get_group_rank(g, group_teams[g], TEAMS, group_fixtures[g])
g_sim

{'A': ({1: 'ECU', 2: 'NED', 3: 'QAT', 4: 'SEN'}, 'a-c'),
 'B': ({1: 'USA', 2: 'ENG', 3: 'IRN', 4: 'WAL'}, 'a-c'),
 'C': ({1: 'ARG', 2: 'POL', 3: 'KSA', 4: 'MEX'}, 'a-c'),
 'D': ({1: 'FRA', 2: 'DEN', 3: 'TUN', 4: 'AUS'}, 'a-c'),
 'E': ({1: 'CRC', 2: 'GER', 3: 'JPN', 4: 'ESP'}, 'd-f'),
 'F': ({1: 'CRO', 2: 'BEL', 3: 'CAN', 4: 'MAR'}, 'a-c'),
 'G': ({1: 'SRB', 2: 'BRA', 3: 'CMR', 4: 'SUI'}, 'a-c'),
 'H': ({1: 'URU', 2: 'GHA', 3: 'KOR', 4: 'POR'}, 'a-c')}

In [44]:
group_fixtures

{'A': [{'date': 'Nov 21',
   'id': 1,
   't1': 'QAT',
   't2': 'ECU',
   't1_goals': 1,
   't2_goals': 1},
  {'date': 'Nov 21',
   'id': 2,
   't1': 'SEN',
   't2': 'NED',
   't1_goals': 3,
   't2_goals': 0},
  {'date': 'Nov 25',
   'id': 18,
   't1': 'QAT',
   't2': 'SEN',
   't1_goals': 1,
   't2_goals': 2},
  {'date': 'Nov 25',
   'id': 19,
   't1': 'NED',
   't2': 'ECU',
   't1_goals': 0,
   't2_goals': 2},
  {'date': 'Nov 29',
   'id': 35,
   't1': 'ECU',
   't2': 'SEN',
   't1_goals': 2,
   't2_goals': 0},
  {'date': 'Nov 29',
   'id': 36,
   't1': 'NED',
   't2': 'QAT',
   't1_goals': 2,
   't2_goals': 2}],
 'B': [{'date': 'Nov 21',
   'id': 3,
   't1': 'ENG',
   't2': 'IRN',
   't1_goals': 0,
   't2_goals': 1},
  {'date': 'Nov 21',
   'id': 4,
   't1': 'USA',
   't2': 'WAL',
   't1_goals': 0,
   't2_goals': 0},
  {'date': 'Nov 25',
   'id': 17,
   't1': 'WAL',
   't2': 'IRN',
   't1_goals': 1,
   't2_goals': 2},
  {'date': 'Nov 25',
   'id': 20,
   't1': 'ENG',
   't2': 'USA',


In [40]:
# Obtain group tables
groups=dict()
for k, g in group_teams.items():
    groups[k]=list()
    for team in g:
        t = TEAMS[team]
        groups[k].append([t.name, t.points, t.goals_scored - t.goals_received, t.goals_scored])
    
    group_df = pd.DataFrame(groups[k], columns=['team','pts','gdf', 'gs']).sort_values(['pts','gdf','gs'], ascending=[False, False, False])
    group_df.index = [1,2,3,4]
    print(k)
    display(group_df)

A


,team,pts,gdf,gs
1,QAT,7,7,8
2,ECU,3,0,2
3,NED,2,-3,1
4,SEN,2,-4,2


B


,team,pts,gdf,gs
1,WAL,9,5,5
2,USA,4,-2,3
3,ENG,3,1,4
4,IRN,1,-4,1


C


...

,team,pts,gdf,gs
1,BEL,7,4,5
2,CAN,6,3,6
3,MAR,3,-5,3
4,CRO,1,-2,5


G


,team,pts,gdf,gs
1,CMR,9,3,5
2,BRA,4,1,5
3,SRB,2,-1,4
4,SUI,1,-3,3


H


,team,pts,gdf,gs
1,URU,6,5,7
2,GHA,6,-1,3
3,POR,4,1,5
4,KOR,1,-5,2


In [213]:
TEAMS = create_teams()
scores = simulate_group_stage(TEAMS).to_dict('index')

group_teams = dict()
group_fixtures = dict()
for g in GROUPS:
    teams = db.execute('SELECT code FROM teams WHERE "group"=?', g)
    group_teams[g] = [team['code'] for team in teams]
    fixtures = db.execute('SELECT * FROM fixtures WHERE team1 IN (SELECT code FROM teams WHERE "group"=?) ORDER BY date;', g)
    group_fixtures[g] = [{'date': datetime.strptime(match['date'], "%Y-%m-%d").strftime("%b %d"),
                             'id':match['match'], 't1': match['team1'], 't2' :match['team2'], 
                             't1_goals': scores[match['match']]['t1_goals'],
                             't2_goals': scores[match['match']]['t2_goals']} for match in fixtures]
print(group_teams)
#group_fixtures

{'A': ['SEN', 'QAT', 'NED', 'ECU'], 'B': ['IRN', 'ENG', 'USA', 'WAL'], 'C': ['ARG', 'KSA', 'MEX', 'POL'], 'D': ['DEN', 'TUN', 'FRA', 'AUS'], 'E': ['GER', 'JPN', 'ESP', 'CRC'], 'F': ['MAR', 'CRO', 'BEL', 'CAN'], 'G': ['SUI', 'CMR', 'BRA', 'SRB'], 'H': ['URU', 'KOR', 'POR', 'GHA']}


In [226]:
crit = 'a-c'
while crit == 'a-c':
    TEAMS = create_teams()
    scores = simulate_group_stage(TEAMS).to_dict('index')

    group_teams = dict()
    group_fixtures = dict()
    for g in GROUPS:
        teams = db.execute('SELECT code FROM teams WHERE "group"=?', g)
        group_teams[g] = [team['code'] for team in teams]
        fixtures = db.execute('SELECT * FROM fixtures WHERE team1 IN (SELECT code FROM teams WHERE "group"=?) ORDER BY date;', g)
        group_fixtures[g] = [{'date': datetime.strptime(match['date'], "%Y-%m-%d").strftime("%b %d"),
                                 'id':match['match'], 't1': match['team1'], 't2' :match['team2'], 
                                 't1_goals': scores[match['match']]['t1_goals'],
                                 't2_goals': scores[match['match']]['t2_goals']} for match in fixtures]
        
        rank, crit = get_group_rank(g, group_teams[g], TEAMS, group_fixtures[g])
        print(crit)

a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
d-f
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
d-f
a-c
a-c
a-c
a-c
d-f
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
d-f
a-c
a-c
a-c
a-c
a-c
a-c
a-c
d-f
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
g-h
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c
a-c


In [218]:
g="H"
get_group_rank(g, group_teams[g], TEAMS, group_fixtures[g])


({1: 'KOR', 2: 'GHA', 3: 'URU', 4: 'POR'}, 'd-f')

In [220]:
g_sim = dict()
for g in GROUPS:
            for i, match in enumerate(fixtures):
                group_fixtures[g][i]['t1_goals'] = scores[match['match']]['t1_goals']
                group_fixtures[g][i]['t2_goals'] = scores[match['match']]['t2_goals']

            g_sim[g] = get_group_rank(g, group_teams[g], TEAMS, group_fixtures[g])

In [224]:
print(g_sim['A'][1])

a-c


In [36]:
TEAM_CODES = db.execute('SELECT code, team FROM teams;')
TEAM_CODES = {team['code']:team['team'] for team in TEAM_CODES}
TEAM_CODES

{'SEN': 'Senegal',
 'QAT': 'Qatar',
 'NED': 'Netherlands',
 'ECU': 'Ecuador',
 'IRN': 'Iran',
 'ENG': 'England',
 'USA': 'United States',
 'WAL': 'Wales',
 'ARG': 'Argentina',
 'KSA': 'Saudi Arabia',
 'MEX': 'Mexico',
 'POL': 'Poland',
 'DEN': 'Denmark',
 'TUN': 'Tunisia',
 'FRA': 'France',
 'AUS': 'Australia',
 'GER': 'Germany',
 'JPN': 'Japan',
 'ESP': 'Spain',
 'CRC': 'Costa Rica',
 'MAR': 'Morocco',
 'CRO': 'Croatia',
 'BEL': 'Belgium',
 'CAN': 'Canada',
 'SUI': 'Switzerland',
 'CMR': 'Cameroon',
 'BRA': 'Brazil',
 'SRB': 'Serbia',
 'URU': 'Uruguay',
 'KOR': 'South Korea',
 'POR': 'Portugal',
 'GHA': 'Ghana'}

In [38]:
help(datetime)

Help on class datetime in module datetime:

class datetime(date)
 |  datetime(year, month, day[, hour[, minute[, second[, microsecond[,tzinfo]]]]])
 |  
 |  The year, month and day arguments are required. tzinfo may be None, or an
 |  instance of a tzinfo subclass. The remaining arguments may be ints.
 |  
 |  Method resolution order:
 |      datetime
 |      date
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __radd__(self, value

In [48]:
date=datetime.strptime("12-4-2022","%m-%d-%Y")

In [50]:
date.strftime("%b %d")

'Dec 04'

In [88]:
TEAMS

{'SEN': <__main__.Team at 0x14ca93310>,
 'QAT': <__main__.Team at 0x14ca93100>,
 'NED': <__main__.Team at 0x14ca39a90>,
 'ECU': <__main__.Team at 0x14ca39490>,
 'IRN': <__main__.Team at 0x14ae7c760>,
 'ENG': <__main__.Team at 0x14b204430>,
 'USA': <__main__.Team at 0x14aebd5b0>,
 'WAL': <__main__.Team at 0x14ca83850>,
 'ARG': <__main__.Team at 0x14ca835e0>,
 'KSA': <__main__.Team at 0x14ca834f0>,
 'MEX': <__main__.Team at 0x14ca83160>,
 'POL': <__main__.Team at 0x14ca837f0>,
 'DEN': <__main__.Team at 0x14ca839d0>,
 'TUN': <__main__.Team at 0x14ca83940>,
 'FRA': <__main__.Team at 0x14ca83a60>,
 'AUS': <__main__.Team at 0x14ca83910>,
 'GER': <__main__.Team at 0x14ca832b0>,
 'JPN': <__main__.Team at 0x14ca83b20>,
 'ESP': <__main__.Team at 0x14ca834c0>,
 'CRC': <__main__.Team at 0x14ca83610>,
 'MAR': <__main__.Team at 0x14ca83ac0>,
 'CRO': <__main__.Team at 0x14ca833d0>,
 'BEL': <__main__.Team at 0x14ca830a0>,
 'CAN': <__main__.Team at 0x14ca83be0>,
 'SUI': <__main__.Team at 0x14ca83250>,


In [60]:
for x1, x2 in zip(group[::2], group[1::2]):
    print(x1, x2)

0 1
2 3
4 5
6 7
